In [3]:
import os
from os import listdir
from os.path import isfile, join
from shutil import copy2
#from tqdm import tqdm #pip install tqdm
import progressbar #pip install progressbar2

#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import numpy as np
import pandas as pd
import glob,cv2
import sys,argparse
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support as score

tf.logging.set_verbosity(tf.logging.ERROR)

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
def predict(filename):
    image_size=128
    num_channels=3
    images = []
    # Reading the image using OpenCV
    image = cv2.imread(filename)
    # Resizing the image to our desired size and preprocessing will be done exactly as done during training
    image = cv2.resize(image, (image_size, image_size),0,0, cv2.INTER_LINEAR)
    images.append(image)
    images = np.array(images, dtype=np.uint8)
    images = images.astype('float32')
    images = np.multiply(images, 1.0/255.0) 
    #The input to the network is of shape [None image_size image_size num_channels]. Hence we reshape.
    x_batch = images.reshape(1, image_size,image_size,num_channels)

    ## Let us restore the saved model 
    sess = tf.Session()
    # Step-1: Recreate the network graph. At this step only graph is created.
    saver = tf.train.import_meta_graph('benign-malicious-model.meta')
    # Step-2: Now let's load the weights saved using the restore method.
    saver.restore(sess, tf.train.latest_checkpoint('./'))

    # Accessing the default graph which we have restored
    graph = tf.get_default_graph()

    # Now, let's get hold of the op that we can be processed to get the output.
    # In the original network y_pred is the tensor that is the prediction of the network
    y_pred = graph.get_tensor_by_name("y_pred:0")

    ## Let's feed the images to the input placeholders
    x= graph.get_tensor_by_name("x:0") 
    y_true = graph.get_tensor_by_name("y_true:0") 
    y_test_images = np.zeros((1, 2)) 

    ### Creating the feed_dict that is required to be fed to calculate y_pred 
    feed_dict_testing = {x: x_batch, y_true: y_test_images}
    result=sess.run(y_pred, feed_dict=feed_dict_testing)
    # result is of this format [probabiliy_of_benign probability_of_malicious]
    #print(result)
    return np.argmax(result, axis=1)[0]

In [5]:
column = ['file','gt_result','jh_result']
df_jh_part = pd.DataFrame(columns=column)
jar_loc = ['testing_data/benign/',
           'testing_data/malicious/']
for loc in jar_loc:
    index = jar_loc.index(loc)
    onlyfiles = [f for f in listdir(loc) if isfile(join(loc, f))]
    #print(len(onlyfiles))
    progress = progressbar.ProgressBar()
    for file,i in zip(onlyfiles, progress(range(len(onlyfiles)))):
        df_jh_part = df_jh_part.append({'file':file,
                                        'gt_result':index,
                                        'jh_result':predict(loc+file)},
                                       ignore_index=True)

 99% (417 of 418) |##################### | Elapsed Time: 1:38:27 ETA:   0:00:19

In [6]:
df_jh_part.shape

(781, 3)

In [7]:
gt = df_jh_part['gt_result'].tolist()
jh = df_jh_part['jh_result'].tolist()

precision, recall, fscore, support = score(gt, jh)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support),'\n')

print(classification_report(gt, jh))
print('accuracy', accuracy_score(gt, jh))

precision: [0.92857143 0.97022333]
recall: [0.96694215 0.9354067 ]
fscore: [0.94736842 0.95249695]
support: [363 418] 

              precision    recall  f1-score   support

           0       0.93      0.97      0.95       363
           1       0.97      0.94      0.95       418

    accuracy                           0.95       781
   macro avg       0.95      0.95      0.95       781
weighted avg       0.95      0.95      0.95       781

accuracy 0.9500640204865557
